# 1. 反向传播的定义与价值

在梯度下降的最初，我们需要先找出坐标点对应的梯度向量。梯度向量是各个自变量求偏导后的表达式再带入坐标点计算出来的，这一步骤中，最大的难点在于如何获得梯度向量的表达式——也就是损失函数对各个自变量求偏导后的表达式。在单层神经网络，例如逻辑回归中，我们有如下计算：

![Alt text](image-13.png)

其中 BCELoss 是二分类交叉熵损失函数。在这个计算图中，从左向右计算的过程就是正向传播，因此进行次计算后，我们会获得所有节点上的张量的值（z， sigma 以及 loss）。根据梯度向量的定义，在这个计算过程中我们要求的是损失函数对 W 的导数，所以求导过程需要涉及到的链路如下：

![Alt text](image-14.png)

用公式来表示则为在以下式子上求解对 W 的导数:

$$
\begin{aligned}
\text { Loss } & =-\sum_{i=1}^m\left(y_i * \ln \left(\sigma_i\right)+\left(1-y_i\right) * \ln \left(1-\sigma_i\right)\right) \\
& =-\sum_{i=1}^m\left(y_i * \ln \left(\frac{1}{1+e^{-X_i w}}\right)+\left(1-y_i\right) * \ln \left(1-\frac{1}{1+e^{-X_i w}}\right)\right)
\end{aligned}
$$

可以看出，已经很复杂了。

更夸张的是，在双层、各层激活函数都是 sigmoid 的二分类神经网络上，我们有如下计算流程：

![Alt text](image-15.png)

同样的，进行从左到右的正向传播之后，我们会获得所有节点上的张量。其中涉及到的求导链路如下：

![Alt text](image-16.png)

用公式来表示，对 w(1 → 2)，我们有：

$$
\frac{\partial L o s s}{\partial w^{(1 \rightarrow 2)}}
$$

其中

$$
\begin{aligned}
\text { Loss } & =-\sum_{i=1}^m\left(y_i * \ln \left(\sigma_i^{(2)}\right)+\left(1-y_i\right) * \ln \left(1-\sigma_i^{(2)}\right)\right) \\
& =-\sum_{i=1}^m\left(y_i * \ln \left(\frac{1}{1+e^{-\sigma_i^{(1)} w^{(1 \rightarrow 2)}}}\right)+\left(1-y_i\right) * \ln \left(1-\frac{1}{1+e^{-\sigma_i^{(1)} w^{(1 \rightarrow 2)}}}\right)\right)
\end{aligned}
$$

对 w（0 → 1），我们有：

$$
\frac{\partial L o s s}{\partial w^{(0 \rightarrow 1)}}
$$

其中

$$
\begin{aligned}
\text{Loss} &=- \sum_{i=1}^{m} \left(y_{i} \ln \left(\sigma_{i}^{(2)}\right)+\left(1-y_{i}\right) \ln \left(1-\sigma_{i}^{(2)}\right)\right)\\
&= -\sum_{i=1}^{m} \left(y_{i} \ln \left(\frac{1}{1+e^{-\frac{1}{1+e^{-X_{i} \boldsymbol{w}^{(0 \rightarrow 1)}}} \boldsymbol{w}^{(1 \rightarrow 2)}}}\right) + (1-y_{i}) \ln \left(1-\frac{1}{1+e^{-\frac{1}{1+e^{-X_{i} \boldsymbol{w}^{(0 \rightarrow 1)}}} \boldsymbol{w}^{(1 \rightarrow 2)}}}\right)\right)
\end{aligned}
$$

对于这个需要对这个式子求导，大家感受如何？而这只是一个两层的二分类神经网络，对于复杂神经网络来说，所需要做的求导工作是无法想象的。求导过程的复杂是神经网络历史上的一大难题，直到1986年才被提出的反向传播算法所解决。

**反向传播算法的基石： 链式法则**

当函数之间存在复杂的嵌套关系，并且我们需要从最外层的函数向最内层的自变量求导时，链式法则可以让求导过程变得异常简单

![Alt text](image-17.png)

以双层二分类网络为例，对 w(1 → 2)我们本来需要求解：

$$
\frac{\partial L o s s}{\partial w^{(1 \rightarrow 2)}}
$$

其中

$$
\begin{aligned}
\text { Loss } & =-\sum_{i=1}^m\left(y_i * \ln \left(\sigma_i^{(2)}\right)+\left(1-y_i\right) * \ln \left(1-\sigma_i^{(2)}\right)\right) \\
& =-\sum_{i=1}^m\left(y_i * \ln \left(\frac{1}{1+e^{-\sigma_i^{(1)} w^{(1 \rightarrow 2)}}}\right)+\left(1-y_i\right) * \ln \left(1-\frac{1}{1+e^{-\sigma_i^{(1)} w^{(1 \rightarrow 2)}}}\right)\right)
\end{aligned}
$$

现在，因为Loss是一个内部嵌套了很多函数的函数，我们可以用链式法则将 $\frac{\partial L o s s}{\partial w^{(1 \rightarrow 2)}}$ 拆解为如下结构：

$$
\frac{\partial \text { Loss }}{\partial w^{(1-2)}}=\frac{\partial L(\sigma)}{\partial \sigma} * \frac{\partial \sigma(z)}{\partial z} * \frac{\partial z(w)}{\partial w}
$$

其中，

$$
\begin{aligned}
\frac{\partial L(\sigma)}{\partial \sigma} & =\frac{\partial\left(-\sum_{i=1}^m\left(y_i * \ln \left(\sigma_i\right)+\left(1-y_i\right) * \ln \left(1-\sigma_i\right)\right)\right)}{\partial \sigma} \\
& =\sum_{i=1}^m \frac{\partial\left(-\left(y_i * \ln \left(\sigma_i\right)+\left(1-y_i\right) * \ln \left(1-\sigma_i\right)\right)\right)}{\partial \sigma}
\end{aligned}
$$

求导不影响加和符号，因此暂时不看加和符号：

$$
\begin{aligned}
& =-\left(y * \frac{1}{\sigma}+(1-y) * \frac{1}{1-\sigma} *(-1)\right) \\
& =-\left(\frac{y}{\sigma}+\frac{y-1}{1-\sigma}\right) \\
& =-\left(\frac{y(1-\sigma)+(y-1) \sigma}{\sigma(1-\sigma)}\right) \\
& =-\left(\frac{y-y \sigma+y \sigma-\sigma}{\sigma(1-\sigma)}\right) \\
& =\frac{\sigma-y}{\sigma(1-\sigma)}
\end{aligned}
$$

假设我们已经进行过一次正向传播，那此时的 $ \sigma $ 就是 $ \sigma ^{(2)} $ , y就是真实标签，我们可以很容易计算出 $\frac{\sigma-y}{\sigma(1-\sigma)}$ 的数值。

再来看剩下的两部分：

$$
\begin{aligned}
\frac{\partial \sigma(z)}{\partial z} & =\frac{\partial \frac{1}{1+e^{-z}}}{\partial z} \\
& =\frac{\partial\left(1+e^{-z}\right)^{-1}}{\partial z} \\
& =-1 *\left(1+e^{-z}\right)^{-2} * e^{-z} *(-1) \\
& =\frac{e^{-z}}{\left(1+e^{-z}\right)^2} \\
& =\frac{1+e^{-z}-1}{\left(1+e^{-z}\right)^2} \\
& =\frac{1+e^{-z}}{\left(1+e^{-z}\right)^2}-\frac{1}{\left(1+e^{-z}\right)^2} \\
& =\frac{1}{\left(1+e^{-z}\right)}-\frac{1}{\left(1+e^{-z}\right)^2} \\
& =\frac{1}{\left(1+e^{-z}\right)}\left(1-\frac{1}{\left(1+e^{-z}\right)}\right) \\
& =\sigma(1-\sigma)
\end{aligned}
$$

此时的 $ \sigma $ 还是 $ \sigma ^{(2)} $ 。接着：

$$
\begin{aligned}
\frac{\partial z(w)}{\partial w} & =\frac{\partial \boldsymbol{\sigma}^{(1)} \boldsymbol{w}}{\partial w} \\
& =\boldsymbol{\sigma}^{(\mathbf{1})}
\end{aligned}
$$

对任意一个特征矩阵 W 而言，$\frac{\partial z(w)}{\partial w}$ 的值就等于其对应的输入值，所以如果是对于单层逻辑回归而言，这里的求导结果应该是 x。不过现在我们是对于双层神经网络的输出层而言，所以这个输入就是从中间层传过来的 
$\sigma ^{1}$ 。现在将三个导数公式整合：

$$
\begin{aligned}
\frac{\partial \text { Loss }}{\partial w^{(1-2)}} & =\frac{\partial L(\sigma)}{\partial \sigma} * \frac{\partial \sigma(z)}{\partial z} * \frac{\partial z(w)}{\partial w} \\
& =\frac{\sigma^{(2)}-y}{\sigma^2\left(1-\sigma^{(2)}\right)} * \sigma^{(2)}\left(1-\sigma^{(2)}\right) * \sigma^{(1)} \\
& =\sigma^{(1)}\left(\sigma^{(2)}-y\right)
\end{aligned}
$$

可以发现，将三个偏导数相乘之后，得到的最终的表达式其实非常简答。并且，其中所需要的数据都是我们在正向传播过程中已经计算出来的节点上的张量。同理，我们也可以得到对 W （0 → 1）的导数。本来我们需要求解：

$$
\frac{\partial L o s s}{\partial w^{(0 \rightarrow 1)}}
$$

其中

$$
\begin{aligned}
\text{Loss} &=- \sum_{i=1}^{m} \left(y_{i} \ln \left(\sigma_{i}^{(2)}\right)+\left(1-y_{i}\right) \ln \left(1-\sigma_{i}^{(2)}\right)\right)\\
&= -\sum_{i=1}^{m} \left(y_{i} \ln \left(\frac{1}{1+e^{-\frac{1}{1+e^{-X_{i} \boldsymbol{w}^{(0 \rightarrow 1)}}} \boldsymbol{w}^{(1 \rightarrow 2)}}}\right) + (1-y_{i}) \ln \left(1-\frac{1}{1+e^{-\frac{1}{1+e^{-X_{i} \boldsymbol{w}^{(0 \rightarrow 1)}}} \boldsymbol{w}^{(1 \rightarrow 2)}}}\right)\right)
\end{aligned}
$$

![Alt text](image-18.png)

现在根据链式法则，就有：

  $$
\frac{\partial \text { Loss }}{\partial w^{(0 \rightarrow 1)}}=\frac{\partial L(\sigma)}{\partial \sigma^{(2)}} * \frac{\partial \sigma(z)}{\partial z^{(2)}} * \frac{\partial z(\sigma)}{\partial \sigma^{(1)}} * \frac{\partial \sigma(z)}{\partial z^{(1)}} * \frac{\partial z(w)}{\partial w^{(0 \rightarrow 1)}}
$$

其中前两项是在求解 $\boldsymbol{w}^{1 \rightarrow 2}$ 时求解过的，而后三项的求解结果都显而易见：

$$
\begin{aligned}
& =\left(\sigma^{(2)}-y\right) * \frac{\partial z(\sigma)}{\partial \sigma^{(1)}} * \frac{\partial \sigma(z)}{\partial z^{(1)}} \quad * \frac{\partial z(w)}{\partial w^{(0 \rightarrow 1)}} \\
& =\left(\sigma^{(2)}-y\right) * w^{1 \rightarrow 2} *\left(\sigma^{(1)}\left(1-\sigma^{(1)}\right)\right) * X
\end{aligned}
$$

同样，这个表达式现在变得非常简答，并且，这个表达式中所需要的全部张量，都是我们在正向传播过程中已经计算出来出来储存好的，或者在模型建立之初就设置好的，因此计算 $\boldsymbol{w}^{0 \rightarrow 1}$ 的导数时，无需再重新计算如 $ \sigma ^{（2）} $ 这样的张量，这就为神经网络计算导数节省了时间。**你是否注意到，我们是从右向左，从输出到输入，逐渐往前求解导数的表达式，并且我们所使用的节点上的张量，也是从后向前逐渐用到，这和我们正向传播的过程完全相反。**
这种从右到左，不断使用正向传播中的元素对梯度向量进行计算的方式，就是反向传播。

# 2. PyTorch实现反向传播


In [3]:
import torch

x = torch.tensor(1, requires_grad= True, dtype= torch.float32)
y = torch.tensor(2, requires_grad= True, dtype= torch.float32)
z = x ** 2
sigma = torch.sigmoid(z)
loss = -(y * torch.log(sigma) + (1 -y) * torch.log(1 - sigma))
torch.autograd.grad(loss, sigma)

(tensor(-6.4540),)

In [4]:
(sigma - y)/(sigma * (1 - sigma))

tensor(-6.4540, grad_fn=<DivBackward0>)

In [23]:
import torch
import torch.nn as nn
from torch.nn import Module
from torch.nn import functional as F

torch.manual_seed(420)
X = torch.rand((500, 20), dtype= torch.float32)* 100
y = torch.randint(low= 0, high= 3 ,size=(500,), dtype= torch.float32)
input_ = X.shape[1]
output_ = len(y.unique())

class Model(Module):
    def __init__(self, in_features = 10 , out_features= 2):
        super(Model, self).__init__()
        self.linear_1 = nn.Linear(in_features, 13, bias= False)
        self.linear_2 = nn.Linear(13, 8, bias= False)
        self.out = nn.Linear(8, out_features, bias= True)
    
    def forward(self, X):
        self.sigma_1 = torch.relu(self.linear_1(X))
        sigma_2 = torch.sigmoid(self.linear_2(self.sigma_1))
        zhat = self.out(sigma_2)
        return zhat
        # z1 = self.linear_1(X)
        # sigma_1 = torch.relu(z1)
        # z2 = self.linear_2(sigma_1)
        # sigma_2 = torch.sigmoid(z2)
        # zhat = s               elf.out(sigma_2)
        # return zhat
    
torch.manual_seed(420)
net = Model(input_, output_)
zhat = net.forward(X)

# logsm = nn.LogSoftmax(dim= 1 )
# logsigma = logsm(zhat)
# criterion = nn.NLLLoss()
# loss = criterion(logsigma, y.long())
# loss
criterion = nn.CrossEntropyLoss()
loss = criterion(zhat, y.long())
loss.backward()

torch.Size([13, 20])

In [28]:
net.linear_2.weight.grad

tensor([[ 1.7231e-04, -2.5986e-03, -1.0750e-02, -2.1731e-04, -9.7872e-04,
         -1.0410e-05, -2.0721e-02, -2.0988e-03, -4.9469e-04,  1.0115e-02,
         -1.9620e-03, -2.0976e-02, -1.7871e-03],
        [ 2.5456e-09,  5.6499e-03,  1.0779e-02, -4.7033e-04,  1.3658e-03,
          5.9375e-04,  8.5078e-03,  1.3552e-02,  1.0980e-02,  2.9854e-04,
          2.5680e-05,  7.0893e-03, -2.2526e-04],
        [ 3.1231e-03,  7.4019e-03, -8.6188e-03, -3.5704e-03,  1.0833e-03,
         -5.7953e-04,  1.8855e-02, -3.8350e-03,  9.2142e-03, -3.3749e-03,
          6.2720e-04,  8.2907e-03, -3.1603e-04],
        [ 6.6471e-07, -4.3162e-04, -1.4294e-02, -5.8203e-04,  4.3025e-04,
         -2.9906e-03, -2.1339e-02, -4.7988e-03, -8.6977e-04, -4.8356e-03,
          4.6847e-05, -9.1177e-03, -3.6824e-05],
        [-2.0897e-03,  5.6794e-03,  2.3308e-02,  1.2448e-02,  1.3274e-02,
          3.4002e-05,  5.9595e-02,  4.8599e-02,  1.8097e-02,  1.6389e-02,
          9.5366e-04,  4.3583e-02,  7.3835e-05],
        [-3.403

backward求解出的结果的结构与对应的权重矩阵结构一模一样，因为一个权重矩阵就对应了一个偏导数。

需要说明的一点是，在使用 atutograd 的时候，我们强调了 requires_grad 的用法，但在定义打包好的类以及使用 loss。backward的时候，我们却没有给任何数据定义 requires_grad = True。这是因为：

1. 当使用 nn.Module继承后的类进行正向传播时，我们的权重 W 是自动生成的，在生成时就被自动设置为允许梯度 （requires_grad = True）,所以不需要我们自己去设置

2. 同时，我们观察反向传播过程

![Alt text](image-19.png)

不难发现，我们的特征张量X 与真实标签y都不在反向传播的过程中，但是X与y其实都损失函数计算需要用到的值，在计算图上，这些值都位于叶子节点上，我们在定义损失函数时，并没有告诉损失函数哪些值是自变量，哪些是常数，那 backward函数是怎么判断具体求解哪个对象的梯度的呢？

其实就是靠 requires_grad。首先 backward 值会识别叶子节点，不在叶子节点上的变量是不会被 backward考虑的。对于全部叶子节点来说，只有属性 requires_grad = True的节点，才会被计算。在设置 X与y 时，我们都没有写requires_grad参数，也就是默认让“允许求解梯度”这个选项为 Flase，所以 backward在计算的时候就只会计算关于 W 的那部分。

当然，我们也可以将 X和y或者任何除了权重以及截距的量的 requires_grad打开，一旦我们设置为True，backward就会帮助我们计算 W 导数的同时，也帮我们计算以 X，或者y 为自变量的导数。在正常的梯度下降和反向传播过程中，我们是不需要这些导数的，因此，我们一律不去管 requires_grad的设置，就让它默认为 False,以节约计算资源。当然，如果你的 W 是自己设置的，千万记得一定要设置 requires_grad = True。